In [18]:
pip list

Package                      Version
---------------------------- ---------
absl-py                      1.2.0
aiohttp                      3.8.1
aiosignal                    1.2.0
argon2-cffi                  21.1.0
astunparse                   1.6.3
async-timeout                4.0.2
attrs                        21.2.0
backcall                     0.2.0
bleach                       4.1.0
blis                         0.7.8
boto3                        1.21.39
botocore                     1.24.39
brotlipy                     0.7.0
cachetools                   5.2.0
cairocffi                    1.3.0
catalogue                    2.0.8
certifi                      2022.6.15
cffi                         1.15.0
chardet                      5.0.0
charset-normalizer           2.0.12
clean-text                   0.6.0
click                        8.1.2
convokit                     2.5.3
cryptography                 36.0.0
cycler                       0.11.0
cymem                        2.0.6


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from tqdm import tqdm
import torch.nn.functional as F
from gensim.models import Word2Vec
from sklearn import metrics
from sklearn.metrics import classification_report
import itertools
import random

In [20]:
device = 'cuda'

In [21]:
class BertModel(nn.Module):
    def __init__(self, voc_size:int=30000, seq_len: int=512, d_model: int=128, d_ff:int=3072, pad_idx: int=1,
                num_encoder: int=12, num_heads: int=12, n_layers=6, dropout: float=0.1):
        super(BertModel, self).__init__()
        self.pad_idx = pad_idx
        self.layers = nn.ModuleList([nn.TransformerEncoderLayer(d_model=d_model, nhead=8, batch_first=True) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.classifier = nn.Linear(d_model, 1)
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        output = input
        for layer in self.layers:
            output = layer(output)
        output = output.mean(dim=1, keepdim=True)
        h_pooled = self.activ1(self.fc(output)) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]
        return logits_clsf # [B, S, D_model]

In [22]:
bert = BertModel()

In [23]:
input = torch.FloatTensor([[0]*128]*3)

In [6]:
input = input.unsqueeze(dim=0)


In [7]:
input.shape

torch.Size([1, 3, 128])

In [8]:
bert(input)

tensor([[[0.1369]]], grad_fn=<ViewBackward0>)

In [24]:
def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    
    
    ## Method 2 using ravel()
    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    n=10
    preds = [round(i) for i in preds.ravel()]
    acc = metrics.accuracy_score(labels.ravel(), preds)
    f1 = metrics.f1_score(labels.ravel(), preds)
    preds = [preds[i*n:(i+1)*n] for i in range(len(preds)//6)]
    #target_names = list(genre_list.keys())
    #print(classification_report(labels,preds, target_names=target_names))
    return {"auc_micro": auc_micro, "acc" : acc, "f1" : f1, 'pred' : preds}

In [25]:
def train_one_epoch(dataloader, model, optimizer, device, loss_fn):
    model.train()
    tk0 = tqdm(dataloader, total=len(dataloader))
    total_loss = 0.0
    
    for source_tensor, label in tk0:
        optimizer.zero_grad()
        output = model(source_tensor.to(device))
        loss = loss_fn(output.view(-1, output.size(-1)), torch.FloatTensor([label]).to(device))
        
        total_loss += loss
        loss.backward()
        optimizer.step()
        
    avg_train_loss = total_loss / len(dataloader) 
    print(" Average training loss: {0:.2f}".format(avg_train_loss))  

def test_one_epoch(testset, model, loss_fn):
    model.eval()
    tk0 = tqdm(testset, total=len(testset))
    targets = []
    outputs = []
    total_loss = 0.0
    with torch.no_grad():
        for source_tensor, label in tk0:
            label = torch.FloatTensor([label])
            output = model(source_tensor.to(device))
            loss = loss_fn(output.view(-1, output.size(-1)), label.to(device))
            total_loss += loss
            targets.extend(label)
            outputs.extend(torch.sigmoid(output[0]).to('cpu'))
    avg_valid_loss = total_loss / len(testset) 
    score = log_metrics(outputs, targets)
    print(" Average valid loss: {0:.2f}".format(avg_valid_loss))  
    print('AUC_SCORE: ', score["auc_micro"], " acc: ", score["acc"], "f1: ", score["f1"])
    return score, avg_valid_loss
    
def fit(model, train_dataloader, valid_dataloader=None,test_dataloader=None, EPOCHS=3, lr=0.000002):
    loss_fn = nn.BCEWithLogitsLoss() #ignore padding
    optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
    for i in range(EPOCHS):
        print(f"EPOCHS:{i+1}")
        print('TRAIN')
        train_one_epoch(train_dataloader, model, optimizer, device, loss_fn)
        auc, loss = test_one_epoch(valid_dataloader,model, loss_fn)
        if i == 0:
            torch.save(model,'rnn_best.model')
            best_loss = loss
        if  loss<best_loss:
            best_loss = loss
            print(i,' EPOCH BEST MODEL!')
            torch.save(model,'rnn_best.model')
    best_model = torch.load('rnn_best.model')
    auc, loss  = test_one_epoch(test_dataloader,best_model, loss_fn)
    print('BEST MODEL')
    print('AUC_SCORE: ', auc["auc_micro"], " acc: ", auc["acc"], "f1: ", auc["f1"])
    return auc

In [26]:
from gensim.models import Word2Vec

In [27]:
import json

In [28]:
with open('incoherence.json', "r") as json_file:
    incoherence = json.load(json_file)

# 평균 임베딩 

In [215]:
import pickle
with open('emb_avg.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [233]:
type(movie_emb[movie_name][21][0])

numpy.float64

In [234]:
correct_triple = []
wrong_triple = []
nomissing_double = []
missing_double = []
path1 = 'word2vec_model_noscene/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
for filename in filenames:
    movie_name = filename[:-14]
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    if movie_name not in set(incoherence['correct_triple'].keys()):
        continue
    for scene in incoherence['correct_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,128)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if str(movie_emb[movie_name][s-1][0]) != 'nan':
                embs[0][i] = torch.from_numpy(movie_emb[movie_name][s-1])
            else:
                passs = 1
                break
        if not passs:        
            correct_triple.append([embs,[0]])
    
    for scene in incoherence['wrong_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,128)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if str(movie_emb[movie_name][s-1][0]) != 'nan':
                embs[0][i] = torch.from_numpy(movie_emb[movie_name][s-1])
            else:
                passs = 1
                break
        if not passs:     
            wrong_triple.append([embs,[1]])
    
    for scene in incoherence['nomissing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,128)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if str(movie_emb[movie_name][s-1][0]) != 'nan':
                embs[0][i] = torch.from_numpy(movie_emb[movie_name][s-1])
            else:
                passs = 1
                break
        if not passs:     
            nomissing_double.append([embs,[0]])
        
    for scene in incoherence['missing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,128)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if str(movie_emb[movie_name][s-1][0]) != 'nan':
                embs[0][i] = torch.from_numpy(movie_emb[movie_name][s-1])
            else:
                passs = 1
                break
        if not passs:     
            missing_double.append([embs,[1]])


In [235]:
print(len(correct_triple))
print(len(wrong_triple))
print(len(nomissing_double))
print(len(missing_double))

50264
47069
51224
51276


In [236]:
random.Random(4).shuffle(correct_triple)
random.Random(4).shuffle(wrong_triple)
train_data3 = correct_triple[:35000]+wrong_triple[:35000]
valid_data3 = correct_triple[35000:40000]+wrong_triple[35000:40000]
test_data3 = correct_triple[40000:]+wrong_triple[40000:]
random.Random(4).shuffle(train_data3)
random.Random(4).shuffle(valid_data3)
random.Random(4).shuffle(test_data3)

In [237]:
bert3 = BertModel().to('cuda')
score = fit(bert3, train_data3,valid_data3,test_data3,EPOCHS=20)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████| 70000/70000 [09:45<00:00, 119.65it/s]


 Average training loss: 0.69


100%|███████████████████████████████████| 10000/10000 [00:09<00:00, 1085.41it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.5064958389880849  acc:  0.5138 f1:  0.31308279174908166
EPOCHS:2
TRAIN


...

 Average training loss: 0.60


100%|████████████████████████████████████| 10000/10000 [00:12<00:00, 789.97it/s]


 Average valid loss: 0.77
AUC_SCORE:  0.5396481962338724  acc:  0.5276 f1:  0.4542513863216266


100%|███████████████████████████████████| 17333/17333 [00:16<00:00, 1081.38it/s]

 Average valid loss: 0.69
AUC_SCORE:  0.5342401877851181  acc:  0.525760110771361 f1:  0.47888931152529485
BEST MODEL
AUC_SCORE:  0.5342401877851181  acc:  0.525760110771361 f1:  0.47888931152529485
{'auc_micro': 0.5342401877851181, 'acc': 0.525760110771361, 'f1': 0.47888931152529485, 'pred': [[1, 0, 0, 0, 1, 0, 1, 0, 1, 1], [0, 0, 1, 1, 0, 1, 1, 0, 0, 1], [0, 1, 1, 1, 0, 1, 0, 0, 1, 0], [1, 0, 0, 1, 0, 1, 0, 1, 0, 1], [0, 0, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 1, 1, 1, 0, 0, 1, 0, 1], [1, 1, 1, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 1, 0, 1, 0, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 0, 1, 0, 1, 0, 1], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 1, 1, 0, 0, 1], [1, 1, 0, 0, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 1, 1, 1, 1, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 0], [0, 0, 1, 1, 1, 1, 0, 1, 0, 0], [1, 1, 1, 0, 1, 1, 1, 1, 0, 1], [1, 1, 1, 0, 1, 1, 1, 0, 1, 1], [

In [240]:
random.Random(4).shuffle(nomissing_double)
random.Random(4).shuffle(missing_double)
train_data4 = nomissing_double[:35000]+missing_double[:35000]
valid_data4 = nomissing_double[35000:40000]+missing_double[35000:40000]
test_data4 = nomissing_double[40000:]+missing_double[40000:]
random.Random(4).shuffle(train_data4)
random.Random(4).shuffle(valid_data4)
random.Random(4).shuffle(test_data4)

In [ ]:
bert4 = BertModel().to('cuda')
score = fit(bert4, train_data4,valid_data4,test_data4,EPOCHS=20)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████| 70000/70000 [10:34<00:00, 110.24it/s]


 Average training loss: 0.69


100%|████████████████████████████████████| 10000/10000 [00:18<00:00, 539.26it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.5121737243855724  acc:  0.5167 f1:  0.2526673882789547
EPOCHS:2
TRAIN


...

100%|████████████████████████████████████| 70000/70000 [09:18<00:00, 125.37it/s]


 Average training loss: 0.69


100%|███████████████████████████████████| 10000/10000 [00:09<00:00, 1056.89it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.5374623866130412  acc:  0.5281 f1:  0.5019525065963061
4  EPOCH BEST MODEL!
EPOCHS:6
TRAIN


 32%|███████████▍                        | 22149/70000 [02:57<06:34, 121.29it/s]

# edge2vec

In [242]:
correct_triple = []
wrong_triple = []
nomissing_double = []
missing_double = []
path1 = 'word2vec_model_edge/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
for filename in filenames:
    movie_name = filename[:-14]
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    if movie_name not in set(incoherence['correct_triple'].keys()):
        continue
    for scene in incoherence['correct_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,128)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if str(movie_emb[movie_name][s-1][0]) != 'nan':
                embs[0][i] = torch.from_numpy(movie_emb[movie_name][s-1])
            else:
                passs = 1
                break
        if not passs:        
            correct_triple.append([embs,[0]])
    
    for scene in incoherence['wrong_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,128)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if str(movie_emb[movie_name][s-1][0]) != 'nan':
                embs[0][i] = torch.from_numpy(movie_emb[movie_name][s-1])
            else:
                passs = 1
                break
        if not passs:     
            wrong_triple.append([embs,[1]])
    
    for scene in incoherence['nomissing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,128)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if str(movie_emb[movie_name][s-1][0]) != 'nan':
                embs[0][i] = torch.from_numpy(movie_emb[movie_name][s-1])
            else:
                passs = 1
                break
        if not passs:     
            nomissing_double.append([embs,[0]])
        
    for scene in incoherence['missing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,128)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if str(movie_emb[movie_name][s-1][0]) != 'nan':
                embs[0][i] = torch.from_numpy(movie_emb[movie_name][s-1])
            else:
                passs = 1
                break
        if not passs:     
            missing_double.append([embs,[1]])

In [243]:
random.Random(4).shuffle(correct_triple)
random.Random(4).shuffle(wrong_triple)
train_data5 = correct_triple[:35000]+wrong_triple[:35000]
valid_data5 = correct_triple[35000:40000]+wrong_triple[35000:40000]
test_data5 = correct_triple[40000:]+wrong_triple[40000:]
random.Random(4).shuffle(train_data5)
random.Random(4).shuffle(valid_data5)
random.Random(4).shuffle(test_data5)

In [245]:
bert5 = BertModel().to('cuda')
score = fit(bert5, train_data5,valid_data5,test_data5,EPOCHS=20)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████| 70000/70000 [10:02<00:00, 116.14it/s]


 Average training loss: 0.69


100%|████████████████████████████████████| 10000/10000 [00:13<00:00, 745.65it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.5016191676268636  acc:  0.5135 f1:  0.008963128946832348
EPOCHS:2
TRAIN


...

 Average training loss: 0.64


100%|███████████████████████████████████| 10000/10000 [00:09<00:00, 1066.71it/s]


 Average valid loss: 0.72
AUC_SCORE:  0.5494412576657461  acc:  0.535 f1:  0.5303978994142596


100%|███████████████████████████████████| 17333/17333 [00:16<00:00, 1076.47it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.5326423205174561  acc:  0.5244908555933768 f1:  0.40232052211747643
BEST MODEL
AUC_SCORE:  0.5326423205174561  acc:  0.5244908555933768 f1:  0.40232052211747643
{'auc_micro': 0.5326423205174561, 'acc': 0.5244908555933768, 'f1': 0.40232052211747643, 'pred': [[0, 0, 1, 0, 0, 0, 1, 1, 1, 0], [0, 0, 1, 1, 0, 1, 0, 0, 1, 0], [0, 1, 1, 1, 0, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 1, 0, 1, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 0, 1, 1, 0, 0, 1], [0, 1, 0, 0, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 0, 0, 1, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 1, 1], [0, 1, 1, 1, 0, 0, 0, 1, 1, 0], [1, 0, 1, 1, 1, 1, 0, 0, 1, 0], [0, 0, 0, 1, 0, 1, 1, 1, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0], [1, 0, 1, 0, 0, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0, 0, 0, 1], [1, 0, 0, 0, 1, 1, 1, 1, 0, 0]

In [246]:
random.Random(4).shuffle(nomissing_double)
random.Random(4).shuffle(missing_double)
train_data6 = nomissing_double[:35000]+missing_double[:35000]
valid_data6 = nomissing_double[35000:40000]+missing_double[35000:40000]
test_data6 = nomissing_double[40000:]+missing_double[40000:]
random.Random(4).shuffle(train_data6)
random.Random(4).shuffle(valid_data6)
random.Random(4).shuffle(test_data6)

In [247]:
bert6 = BertModel().to('cuda')
score = fit(bert6, train_data6,valid_data6,test_data6,EPOCHS=20)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████| 70000/70000 [09:21<00:00, 124.70it/s]


 Average training loss: 0.69


100%|███████████████████████████████████| 10000/10000 [00:09<00:00, 1061.48it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.5040190761301017  acc:  0.5136 f1:  0.018563357546408393
EPOCHS:2
TRAIN


...

 Average training loss: 0.64


100%|███████████████████████████████████| 10000/10000 [00:09<00:00, 1093.86it/s]


 Average valid loss: 0.72
AUC_SCORE:  0.55048328632862  acc:  0.5415 f1:  0.5211488250652742


100%|███████████████████████████████████| 10000/10000 [00:08<00:00, 1122.66it/s]

 Average valid loss: 0.69
AUC_SCORE:  0.5183098294167306  acc:  0.5163 f1:  0.18910310142497902
BEST MODEL
AUC_SCORE:  0.5183098294167306  acc:  0.5163 f1:  0.18910310142497902
{'auc_micro': 0.5183098294167306, 'acc': 0.5163, 'f1': 0.18910310142497902, 'pred': [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0

# dim 384

In [29]:
correct_triple = []
wrong_triple = []
nomissing_double = []
missing_double = []
path1 = 'word2vec_model_char/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
for filename in filenames:
    movie_name = filename[:-14]
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    if movie_name not in set(incoherence['correct_triple'].keys()):
        continue
    for scene in incoherence['correct_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,384)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:        
            correct_triple.append([embs,[0]])
    
    for scene in incoherence['wrong_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,384)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:     
            wrong_triple.append([embs,[1]])
    
    for scene in incoherence['nomissing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,384)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:     
            nomissing_double.append([embs,[0]])
        
    for scene in incoherence['missing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,384)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:     
            missing_double.append([embs,[1]])


In [30]:
random.Random(4).shuffle(correct_triple)
random.Random(4).shuffle(wrong_triple)
train_data7 = correct_triple[:35000]+wrong_triple[:35000]
valid_data7 = correct_triple[35000:40000]+wrong_triple[35000:40000]
test_data7 = correct_triple[40000:]+wrong_triple[40000:]
random.Random(4).shuffle(train_data7)
random.Random(4).shuffle(valid_data7)
random.Random(4).shuffle(test_data7)

In [16]:
bert7 = BertModel(d_model=384).to('cuda')
score = fit(bert7, train_data7,valid_data7,test_data7,EPOCHS=10)
print(score)

EPOCHS:1
TRAIN


100%|█████████████████████████████████████| 70000/70000 [14:33<00:00, 80.09it/s]


 Average training loss: 0.69


100%|████████████████████████████████████| 10000/10000 [00:11<00:00, 871.56it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.53696652  acc:  0.5235 f1:  0.5884080504448476
EPOCHS:2
TRAIN


...

 Average training loss: 0.65


100%|████████████████████████████████████| 10000/10000 [00:12<00:00, 817.75it/s]


 Average valid loss: 0.72
AUC_SCORE:  0.53668214  acc:  0.5262 f1:  0.5869944211994421
EPOCHS:8
TRAIN


 80%|█████████████████████████████▌       | 55882/70000 [11:56<03:00, 78.01it/s]


KeyboardInterrupt: 

In [14]:
random.Random(4).shuffle(nomissing_double)
random.Random(4).shuffle(missing_double)
train_data8 = nomissing_double[:35000]+missing_double[:35000]
valid_data8 = nomissing_double[35000:40000]+missing_double[35000:40000]
test_data8 = nomissing_double[40000:]+missing_double[40000:]
random.Random(4).shuffle(train_data8)
random.Random(4).shuffle(valid_data8)
random.Random(4).shuffle(test_data8)

In [15]:
bert8 = BertModel(d_model=384).to('cuda')
score = fit(bert8, train_data8,valid_data8,test_data8,EPOCHS=30)
print(score)

EPOCHS:1
TRAIN


100%|█████████████████████████████████████| 70000/70000 [14:25<00:00, 80.84it/s]


 Average training loss: 0.70


100%|████████████████████████████████████| 10000/10000 [00:12<00:00, 800.77it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.52849848  acc:  0.5203 f1:  0.5828332898512915
EPOCHS:2
TRAIN


  2%|▊                                     | 1506/70000 [00:19<15:01, 75.95it/s]


KeyboardInterrupt: 

# extra+emb

In [41]:
correct_triple = []
wrong_triple = []
nomissing_double = []
missing_double = []
path1 = 'word2vec_model_extra+emb/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
for filename in filenames:
    movie_name = filename[:-14]
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    if movie_name not in set(incoherence['correct_triple'].keys()):
        continue
    for scene in incoherence['correct_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,384)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:        
            correct_triple.append([embs,[0]])
    
    for scene in incoherence['wrong_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,384)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:     
            wrong_triple.append([embs,[1]])
    
    for scene in incoherence['nomissing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,384)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:     
            nomissing_double.append([embs,[0]])
        
    for scene in incoherence['missing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,384)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:     
            missing_double.append([embs,[1]])


In [42]:
random.Random(4).shuffle(correct_triple)
random.Random(4).shuffle(wrong_triple)
train_data7 = correct_triple[:35000]+wrong_triple[:35000]
valid_data7 = correct_triple[35000:40000]+wrong_triple[35000:40000]
test_data7 = correct_triple[40000:]+wrong_triple[40000:]
random.Random(4).shuffle(train_data7)
random.Random(4).shuffle(valid_data7)
random.Random(4).shuffle(test_data7)

In [43]:
bert7 = BertModel(d_model=384).to('cuda')
score = fit(bert7, train_data7,valid_data7,test_data7,EPOCHS=10)
print(score)

EPOCHS:1
TRAIN


100%|█████████████████████████████████████| 70000/70000 [15:48<00:00, 73.80it/s]


 Average training loss: 0.69


100%|████████████████████████████████████| 10000/10000 [00:13<00:00, 717.83it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.5215202400000001  acc:  0.5151 f1:  0.5915943737892698
EPOCHS:2
TRAIN


...

 Average training loss: 0.64


100%|███████████████████████████████████| 10000/10000 [00:09<00:00, 1019.07it/s]


 Average valid loss: 0.75
AUC_SCORE:  0.52940992  acc:  0.5165 f1:  0.5612920787587334
EPOCHS:9
TRAIN


  1%|▌                                    | 1046/70000 [00:08<09:39, 118.97it/s]


KeyboardInterrupt: 

In [ ]:
random.Random(4).shuffle(nomissing_double)
random.Random(4).shuffle(missing_double)
train_data8 = nomissing_double[:35000]+missing_double[:35000]
valid_data8 = nomissing_double[35000:40000]+missing_double[35000:40000]
test_data8 = nomissing_double[40000:]+missing_double[40000:]
random.Random(4).shuffle(train_data8)
random.Random(4).shuffle(valid_data8)
random.Random(4).shuffle(test_data8)

In [ ]:
bert8 = BertModel(d_model=384).to('cuda')
score = fit(bert8, train_data8,valid_data8,test_data8,EPOCHS=10)
print(score)

# 최종

In [44]:
correct_triple = []
wrong_triple = []
nomissing_double = []
missing_double = []
path1 = 'word2vec_model_extra+pos/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
for filename in filenames:
    movie_name = filename[:-14]
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    if movie_name not in set(incoherence['correct_triple'].keys()):
        continue
    for scene in incoherence['correct_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,400)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:        
            correct_triple.append([embs,[0]])
    
    for scene in incoherence['wrong_triple'][movie_name]:
        passs = 0
        embs = torch.zeros(1,3,400)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:     
            wrong_triple.append([embs,[1]])
    
    for scene in incoherence['nomissing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,400)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:     
            nomissing_double.append([embs,[0]])
        
    for scene in incoherence['missing_double'][movie_name]:
        passs = 0
        embs = torch.zeros(1,2,400)
        for i,s in enumerate(scene):
            emb_name = base + str(s)
            if emb_name in w2v.wv.index_to_key:
                embs[0][i] = torch.from_numpy(w2v.wv[emb_name])
            else:
                passs = 1
                break
        if not passs:     
            missing_double.append([embs,[1]])


In [45]:
random.Random(4).shuffle(correct_triple)
random.Random(4).shuffle(wrong_triple)
train_data10 = correct_triple[:35000]+wrong_triple[:35000]
valid_data10 = correct_triple[35000:40000]+wrong_triple[35000:40000]
test_data10 = correct_triple[40000:]+wrong_triple[40000:]
random.Random(4).shuffle(train_data10)
random.Random(4).shuffle(valid_data10)
random.Random(4).shuffle(test_data10)

In [46]:
bert10 = BertModel(d_model=400).to('cuda')
score = fit(bert10, train_data10,valid_data10,test_data10,EPOCHS=10)
print(score)

EPOCHS:1
TRAIN


100%|█████████████████████████████████████| 70000/70000 [12:01<00:00, 96.98it/s]


 Average training loss: 0.69


100%|████████████████████████████████████| 10000/10000 [00:10<00:00, 936.27it/s]


 Average valid loss: 0.68
AUC_SCORE:  0.5843455  acc:  0.5683 f1:  0.6439000247463499
EPOCHS:2
TRAIN


...

 Average training loss: 0.47


100%|████████████████████████████████████| 10000/10000 [00:11<00:00, 907.59it/s]


 Average valid loss: 0.65
AUC_SCORE:  0.73319422  acc:  0.6532 f1:  0.6747936984246062


100%|██████████████████████████████████████| 7118/7118 [00:08<00:00, 802.24it/s]


 Average valid loss: 0.63
AUC_SCORE:  0.7107552394944355  acc:  0.6174487215509975 f1:  0.3142785192646689
BEST MODEL
AUC_SCORE:  0.7107552394944355  acc:  0.6174487215509975 f1:  0.3142785192646689
{'auc_micro': 0.7107552394944355, 'acc': 0.6174487215509975, 'f1': 0.3142785192646689, 'pred': [[0, 1, 0, 1, 0, 0, 1, 1, 1, 1], [1, 1, 0, 0, 0, 0, 1, 0, 0, 1], [1, 1, 0, 1, 0, 0, 0, 1, 1, 1], [1, 0, 1, 1, 0, 0, 0, 1, 0, 0], [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 1, 0, 1, 0, 0], [0, 0, 1, 1, 1, 1, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 1, 0], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [0, 0, 1, 0, 0, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [1, 1, 1, 0, 1, 1, 0, 1, 1, 1], [0, 0, 1, 0, 0, 1, 0, 1, 1, 1], [0, 1, 0, 0, 0, 0, 1, 1, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [

In [51]:
random.Random(4).shuffle(nomissing_double)
random.Random(4).shuffle(missing_double)
train_data8 = nomissing_double[:35000]+missing_double[:35000]
valid_data8 = nomissing_double[35000:40000]+missing_double[35000:40000]
test_data8 = nomissing_double[40000:]+missing_double[40000:]
random.Random(4).shuffle(train_data8)
random.Random(4).shuffle(valid_data8)
random.Random(4).shuffle(test_data8)

In [53]:
bert8 = BertModel(d_model=400).to('cuda')
score = fit(bert8, train_data8,valid_data8,test_data8,EPOCHS=10)
print(score)

EPOCHS:1
TRAIN


 13%|█████                                 | 9421/70000 [02:07<13:37, 74.10it/s]


KeyboardInterrupt: 